In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import sys
sys.path.insert(0, "../../Programm/PynPoint")
sys.path.insert(0, "../../Programm/PynGit/species")

import numpy as np
import matplotlib
#matplotlib.use('Agg') #writes results to file, not post in a window
!pip install /content/drive/MyDrive/PynPoint
import pynpoint as pp
from scipy.optimize import curve_fit


# ---------------------------------------------------------------------- variables
# user defined variables
iwa = 10               # inner working angle                	(default = 10)
im_extra_rot = 0       # additional rotation offset         	(default = 0)
im_size_fc_crop  = 200 # Cropping size after data reduction 	(default = 200)
im_size_pca_crop = 200 # Cropping size after pca analysis   	(default = 200)

# fix variables
pixscale = 0.00746     # (FIX) pixel to arcsec scale		    (initially = 0.00746)
ce_r_min = 47          # (FIX) minimum radius of waffle spots	(initially = 47)
ce_l_min = 0.953       # (FIX) smallest observed wavelength	    (initially = 0.953)



# PynPoint SetUp
path_working = '/content/drive/MyDrive/working' #'./working'
path_in      = '/content/drive/MyDrive/reduced' #'./reduced'
path_out     = '/content/drive/MyDrive/results' #'./results'

path_in_data = '/content/drive/MyDrive/reduced' #'./reduced'
path_in_wave = '/content/drive/MyDrive/reduced/wave/wavelength.fits' #'../../thesis_publication/wave/wavelength.fits'
path_in_angl = '/content/drive/MyDrive/reduced/angles/angles.fits' # '../../thesis_publication/angles/angles.fits'

# ---------------------------------------------------------------------- pre run checks

# Add the psf directory to PynPoint SetUp - copy it into the MyDrive directory (like above)
psf = '/content/drive/MyDrive/psf/' #  './psf/' #This is probably useless
input_dir = 'psf'

# check input directory
print(path_in)
assert os.path.isdir(path_in), 'Input directory not valid'

# Create Working directory for this target
if not os.path.isdir(path_working):
    print("Creating directory: %s" % (path_working))
    os.makedirs(path_working)

# Create directory for results
if not os.path.isdir(path_out):
    print("Creating directory: %s" % (path_out))

# ---------------------------------------------------------------------- Pypelin set up

# Set up the Pypelin
pipeline = pp.Pypeline(working_place_in = path_working,
                       input_place_in   = path_in,
                       output_place_in  = path_out)

# --- read in of science frame and wavelength/angle of the frames
im_reading_mod = pp.FitsReadingModule(name_in="im_reading_mod",
                                      input_dir=path_in_data,
                                      image_tag="im",
                                      ifs_data=True)


#FPI
psf_reading_mod = pp.FitsReadingModule(name_in='psf_read',
                                       input_dir=psf, #'psf'
                                       image_tag='psf_test',
                                       ifs_data=True)


im_wavel_mod = pp.WavelengthReadingModule(name_in = "im_wavel_read",
                                          data_tag = "im",
                                          file_name = path_in_wave)

im_angle_mod = pp.ParangReadingModule(name_in = "im_angle_read",
                                      data_tag = "im",
                                      file_name = path_in_angl)



# --- additional preprocessing
im_sort_parang = pp.SortParangModule(name_in = "sort_parang",
                                     image_in_tag = "im",
                                     image_out_tag = "im_sorted")


# --- intermediat printing to check data
im_stack = pp.DerotateAndStackModule(name_in = "im_stack_dict",
                                         image_in_tag = 'im',
                                         image_out_tag = 'im_stack',
                                         derotate = True,
                                         stack = 'median',
                                         extra_rot = 0,
                                         dimension = 'wavelength')

im_stack_write = pp.FitsWritingModule(name_in = "im_stack_write",
                                     file_name= "im_stack.fits",
                                     data_tag = "im_stack")



# --- preparation of the data
im_prep_mod = pp.PSFpreparationModule(name_in = "the_real_preparation_mod",
                                      image_in_tag = "im",
                                      image_out_tag = "im_pca_prep",
                                      mask_out_tag = "mask_sdi_prep",
                                      norm=False,
                                      resize=None,
                                      cent_size=iwa*pixscale,
                                      edge_size=im_size_pca_crop*pixscale)


im_prep_write = pp.FitsWritingModule(name_in = "im_pca_prep",
                                     file_name= "im_pca_prep.fits",
                                     data_tag = "im_pca_prep")


# FPI!
fake_test = pp.FakePlanetModule(name_in = 'fake_test',
                                image_in_tag = "im_pca_prep",
                                psf_in_tag = "psf_test",
                                image_out_tag = "im_fake_test",
                                position = (-1.15, 0),
                                magnitude = np.asarray(np.multiply([-1.0, -1.0, -1.0, -1.0, -1.0, -1.0,
                                                        	-1.0, -1.0, -1.0, -1.0, -1.0, -1.0,
                                                        	-1.0, -1.0, -1.0, -1.0, -1.0, -1.0,
                                                        	-1.0, -1.0, -1.0, -1.0, -1.0, -1.0,
                                                        	-1.0, -1.0, -1.0, -1.0, -1.0, -1.0,
                                                        	-1.0, -1.0, -1.0, -1.0, -1.0, -1.0,
                                                        	-1.0, -1.0, -1.0], 5) ),
                                #magnitude = np.asarray([9., -8., 9., 0., 9., 0.,
                                #                        9., 0., 9., 0., 9., 0.,
                                #                        9., 0., 9., 0., 9., 0.,
                                #                        9., 0., 9., 0., 9., 0.,
                                #                        9., 0., 9., 0., 9., 0.,
                                # The Original           9., 0., 9., 0., 9., 0.,
                                #                        9., 0., 9.]),
                                psf_scaling = 1.,
                                interpolation = 'spline')


fake_write = pp.FitsWritingModule(name_in = "fake_write",
                                  file_name = "fake_test.fits",
                                  data_tag = "im_fake_test")

print('Finished assigning fake_write, define evaluate and go to the next cell')

def evaluate(pipe, p_t, pca):		# for example: evaluate(pipeline, 'ADI',  ([1,2,3]))


    # --- Perform pca reduction
    the_real_test_mod = pp.PcaPsfSubtractionModule(name_in = p_t + '_wave_test_mod',
                                                          images_in_tag = 'im_fake_test',
                                                          reference_in_tag = 'im_pca_prep',
                                                          res_mean_tag = p_t + '_wave_mean',
                                                          res_median_tag = p_t + '_wave_median',
                                                          res_weighted_tag = None,
                                                          res_rot_mean_clip_tag = None,
                                                          res_arr_out_tag  = None,
                                                          basis_out_tag  = None,
                                                          pca_numbers = pca,
                                                          extra_rot = im_extra_rot,
                                                          subtract_mean = True, # makes the image background average at 0
                                                          processing_type = p_t)



    # --- write PCA results
    the_real_median_write_mod = pp.FitsWritingModule(name_in = p_t + "_wave_median_write_mod",
                                                     file_name= p_t + "_wave_median.fits",
                                                     data_tag = p_t + "_wave_median")


    the_real_mean_write_mod = pp.FitsWritingModule(name_in = p_t + "_wave_mean_write_mod",
                                                   file_name = p_t + "_wave_mean.fits",
                                                   data_tag = p_t + "_wave_mean")



    #add all modules
    pipe.add_module(the_real_test_mod)
    pipe.add_module(the_real_median_write_mod)
    pipe.add_module(the_real_mean_write_mod)



# ------------------------------------------------------------------ Pypelin set execution

pipeline.add_module(im_reading_mod)
pipeline.add_module(im_wavel_mod)
pipeline.add_module(im_angle_mod)
pipeline.add_module(im_sort_parang) #vestigual
pipeline.add_module(im_stack)
pipeline.add_module(im_stack_write)
pipeline.add_module(im_prep_mod)
pipeline.add_module(im_prep_write)

# FPI's
pipeline.add_module(psf_reading_mod)
pipeline.add_module(fake_test)
pipeline.add_module(fake_write)

#The Real #Reel?
#evaluate(pipeline, 'SDI',  ([1,2,3,4]))				#(initially = [1,2])
evaluate(pipeline, 'SDI+ADI', ([1], [1]))	#(initially = ([1,2,3,4,5], [1,2,3,4,5])
#evaluate(pipeline, 'ADI+SDI', ([1,2,3], [1,2,3]))	#(initially = ([1,2,3,4,5], [1,2,3,4,5]) and un-commented
#evaluate(pipeline, 'CODI', ([40]))		#(initially = ([100,200,300,400]))


# run the pipeline
pipeline.run()


Processing ./drive/MyDrive/PynPoint
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 103.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 115.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 106.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.1/829.1 kB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 117.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

ImportError: ignored

In [ ]:
# Get the data as an image from a fits *FILE* (the above reduction)

from astropy.io import fits
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt #matplotlib.use('Agg')
from scipy.optimize import curve_fit

path = '/content/drive/MyDrive/' # SADI_wave_median_PCA=1-10_Mag=-1,7_Pos=-1,15.fits #path = '/home/dcpetit/Documents/kuleuven_astronomy/thesis_publication/results/'
#image_file_FPI = path+'SADI_wave_median_PCA=1-10_Mag=-1,7_Pos=-1,15.fits'
image_file_FPI = path+'/results/SDI+ADI_wave_median.fits'
image_data_FPI = fits.getdata(image_file_FPI, ext=0)
print('np.shape(image_data_FPI):', np.shape(image_data_FPI))
x, y = image_data_FPI[0][0][0], image_data_FPI[0][0][1]

# Fucntions to get signal data, given the PCAs and wavelength channel

def set_signalMatrix_1_WLC_1_PCA(image_data, x, y, PCA, WL):
    signal_matrix = np.zeros(200*200).reshape(200,200)
    for i in range(len(x)):
        for j in range(len(y)):
            signal_matrix[i][j] = image_data[PCA][WL][i][j]
    return(signal_matrix)

def set_signalMatrix_1_WLC_2_PCA(image_data, x, y, PCA1, PCA2, WL):
    signal_matrix = np.zeros(200*200).reshape(200,200)
    for i in range(len(x)):
        for j in range(len(y)):
            signal_matrix[i][j] = image_data[PCA1-1][PCA2-1][WL][i][j] # This selects the 33rd Wavelength Detector for ASDI/SADI data with PCA1 = PCA2 = 0
    return(signal_matrix)


In [ ]:
# Determine the location of the exact injected FPI (that gets smeared around)

sep_arcsec = -1.15  # Separation (arcsec)
pixScale = 0.00746  # pixels * arcsec/pixels = arcsec
sep_pixel = -1.15/pixScale
possible_FPI_pixel = [100, -(sep_pixel+100)]
print(sep_pixel)
print(possible_FPI_pixel)

In [ ]:
# Define the CB Parameters... Use the circle from SNR calculations, turn it into a square...
diameter = 5;                    D = diameter
radius = int(diameter/2);        R = radius
location_demo = (80, 131);       L_demo = location_demo
location = (80, 103);            L = location
separation = 50;                 S = separation
decay_factor = 0.9;              DF= decay_factor
angular_squares, radial_squares = 4, 4 # I think there's no code that uses this now - 5 July 23
x_squares, y_squares = 4, 3   # switch for imshow
center_square = [2, 4]
max_brightness, MB = 10, 10   # half_edge = np.sqrt(radius**2 + separation**2) - separation + radius
amp_nonRad_less = 2           # Earlier idea: Use a 2nd Maximum Brightnesses... One for SDI/radial CB spaces... One for all other CB spaces

#amplitude_radial_boost = 2    # Use a 2nd Maximum Brightnesses... One for SDI/radial CB spaces... One for all other CB spaces
#amp_radial_boost = 2          # Make a radial and non-radial, augmentation and diminishing factor
#angular_squares, radial_squares = 4, 4 # I think there's no code that uses this now - 5 July 23


In [ ]:
# Define a fucntion for a checkerboard with no hole for the preseumed exoplanet (just a designated square for the exoplanet)
def make_oscillating_checkerboard_noGap(diameter, x, y, center_square, amplitude, amp_nonRad_less, location, delay_x, delay_y, decay_factor): # amp_radial_boost,
    checkerboard4D = np.zeros([x, y, diameter, diameter])
    checkerboard = np.zeros([200, 200])
    for i in range(x):                          #each row of squares
        for j in range(y):                      #each column of squares
            displacement_square = [ i - (center_square[1]-1) , j - (center_square[0]-1) ] # center_square[index] is backwards for IMSHOW
            decay_exponent = (np.sqrt(displacement_square[0]**2 + displacement_square[1]**2))
            if displacement_square[1] == 0:
                adjustment = 1 #amp_radial_boost
            else:
                adjustment = amp_nonRad_less**(-1)
            if np.sum(displacement_square) % 2 != 0:
                N = 1 * adjustment * decay_factor**decay_exponent   #if displacement_square[0] == 0 and displacement_square[1] == 0:  N = 0   el
            elif np.sum(displacement_square) % 2 == 0:
                N = -1 * adjustment * decay_factor**decay_exponent
            else:
                print('problem in checkerboard creation')
            for k in range(diameter):                  #each x pixel
                for l in range(diameter):              #each y pixel
                    checkerboard4D[i, j, k, l] = N * amplitude * np.cos(np.pi * k / diameter - delay_y) * np.cos(np.pi * l / diameter - delay_x) # it's an l, not a 1 ... #this one is a sine function, could test it out with a gaussian function
            L0, L1 = location[0], location[1]
            checkerboard[i*diameter+L0:(i+1)*diameter+L0, j*diameter+L1:(j+1)*diameter+L1] = checkerboard4D[i,j,:,:]
    return checkerboard, checkerboard4D

In [ ]:
# Make the same plots as above, but use the wavy checkerboard instead...
def add_wavy_checkerboard(observation, checkerboard):
    merged_observation_WavyCheckerboard = np.add(observation, checkerboard)
    return merged_observation_WavyCheckerboard

In [ ]:
# Get the wavelength channel (bin) ranges, and then use their distances to calculate a how much of a phase change should be between each channel
wavelength_channels = [ 954.32282227,  962.62163342,  971.8455533, 981.89907773,  991.62082995, 1001.70638262,
                        1012.66201578, 1023.29215927, 1033.63748036, 1043.90899136, 1054.17332605, 1064.93904728,
                        1075.51906673, 1085.48399237, 1095.372864, 1105.07626751, 1115.18984928, 1127.09242581,
                        1138.79673007, 1149.43864103, 1159.42615783, 1169.31067405, 1179.38222723, 1189.4464981,
                        1199.5333784, 1209.88668205, 1220.04015567, 1229.96503806, 1240.57338913, 1250.6362063,
                        1260.9653672,  1271.4195118,  1281.94965148,1291.7051642,  1300.62445043, 1309.61923874,
                        1318.81874937, 1327.10100597, 1333.64664519] # This comes from the output of psfsubtraction.py which is seen in the reduction cell
#plt.plot(wavelength_channels, 'o')
print("np.shape(wavelength_channels)", np.shape(wavelength_channels))
print(wavelength_channels)
# Given a range (1.5*pi radians), make a descrete sign function that is evaluated in the spacing that the wavelength channels are distributed
wlMin, wlMax, wlDistribution = np.min(wavelength_channels), np.max(wavelength_channels), []
for i in wavelength_channels:
    wlDistribution.append(10*(wlMax - i)/(wlMax - wlMin) - 7.6) # This will replace 39 WLCs with 1 period, which still needs to move ~10 radians so 10rad/1wlDistribution

In [ ]:
# Generate the 39x2 plots, like above, with a gapless checkerboard
WLC_num = 39
PCA_1, PCA_2 = 1, 1
x_squares, y_squares, center_square, location, location_demo = 5, 5, [3, 3], (44, 88), (44, 125)
fig, axs = plt.subplots(nrows=2, ncols=WLC_num, figsize=(8*WLC_num, 14))# sharey=False)
for i in range(WLC_num):
    x, y = image_data_FPI[0][0][0], image_data_FPI[0][0][1]
    if len(np.shape(image_data_FPI)) == 5:
        signalMatrix_FPI = set_signalMatrix_1_WLC_2_PCA(image_data_FPI, x, y, PCA_1, PCA_2, i)   #elif len(np.shape(image_data_FPI)) == 4:   #signalMatrix_FPI = set_signalMatrix_1_WLC_1_PCA(image_data_FPI, x, y, 2, 32)
    else:
        print('Error... image_data_FPI != 5\n') #4 or
    Ydelay = (i * -0.26) + 2.4 # The black starts at the top, goes to the bottom and back to the top (1 cycle), and then to the bottom again (maybe a little more), so about 1.5 cycles, or 1.6-1.7... That would be 2*pi*1.6 = 10 radians over 39 WLC = 0.26rad/WLC...
    Ydelay2= (i * -0.26) + 2.4 # this moves the CB's black (wave) down. The observation's black kinda moves down (or the white up)
    Xdelay = diameter / 4
    wlD = wlDistribution[i]
    max_brightness, amp_radial_boost, amp_nonRad_less, decay_factor = 30, 1, 2, 0.4 # MB=24,18,10 --> too bright
    checkerboard_demo, CB4D_D = make_oscillating_checkerboard_noGap(diameter, x_squares, y_squares, center_square, max_brightness, amp_nonRad_less, location_demo, Xdelay, wlD, decay_factor) # Ydelay2,
    #checkerboard_Ydelay, CB4D_Ydelay = make_oscillating_checkerboard_noGap(diameter, x_squares, y_squares, center_square, max_brightness, amp_nonRad_less, location, Xdelay, Ydelay, decay_factor)
    checkerboard, CB4D = make_oscillating_checkerboard_noGap(diameter, x_squares, y_squares, center_square, max_brightness, amp_nonRad_less, location, Xdelay, wlD, decay_factor) # Ydelay,
    mitigated_signal_D = add_wavy_checkerboard(signalMatrix_FPI, checkerboard_demo)
    #mitigated_signal_Ydelay = add_wavy_checkerboard(signalMatrix_FPI, checkerboard_Ydelay)
    mitigated_signal = add_wavy_checkerboard(signalMatrix_FPI, checkerboard)
    axs[0, i].imshow(mitigated_signal_D, origin='lower', cmap='Blues_r') # im_side[2i] =
    axs[0, i].plot(possible_FPI_pixel[0], possible_FPI_pixel[1], marker='o', color='red')
    axs[1, i].imshow(mitigated_signal, origin='lower', cmap='Blues_r') # im_side[2i+1] =
    axs[1, i].title.set_text(str(i+1)+". min: "+str(np.round(np.min(mitigated_signal), 2)) +". max: "+str(np.round(np.max(mitigated_signal), 2)) )
    #axs[2, i].imshow(mitigated_signal_Ydelay, origin='lower', cmap='Blues_r') # im_side[2i+1] =
plt.show()

In [ ]:
# This is the contents of the modifications in the postproc.py file, which is used to apply the CB to each exposure before de-rotating and stacking:

#...after the elif processing_type == 'CODI': section:


### ### ### Make plots of original res_raw images (WLC = 8, for 32 exposures), and mitigated images ### ### ###

    # Define variables: An original res_raw that preserves the values passed before my edits, and a new matrix for testing (negatives)
    # It seems that the simpler y = x, without np.multiply leads to some weird equivalences and doesn't generate clean new variables
    res_raw_original = np.multiply(res_raw, 1)
    res_raw_neg = np.multiply(res_raw, -1)  # THIS IS IMPORTANT... AFFECTS FINAL OUTCOME, WHICH IT SHOULDN'T - 10 SEP 2023

    shapeResRaw, shapeResRot = np.shape(res_raw), np.shape(res_rot)
    print("\n\n\tbefore modification, np.shape(res_raw):", shapeResRaw)
    print("\n\tbefore modification, np.shape(res_rot):", shapeResRot)

    import matplotlib.pyplot as plt
    diameter = 5;                    D = diameter
    radius = int(diameter/2);        R = radius
    separation = 50;                 S = separation
    decay_factor = 0.9;              DF= decay_factor
    angular_squares, radial_squares = 4, 4 # I think there's no code that uses this now - 5 July 23
    max_brightness, MB = 10, 10   # half_edge = np.sqrt(radius**2 + separation**2) - separation + radius
    amp_nonRad_less = 2
    x_squares, y_squares, center_square, location, location_demo = 5, 5, [3, 3], (44, 88), (44, 125)
    max_brightness, amp_radial_boost, amp_nonRad_less, decay_factor = 30, 1, 2, 0.4 # MB=24,18,10 --> too bright
    wavelength_channels = [ 954.32282227,  962.62163342,  971.8455533, 981.89907773,  991.62082995, 1001.70638262,
                        1012.66201578, 1023.29215927, 1033.63748036, 1043.90899136, 1054.17332605, 1064.93904728,
                        1075.51906673, 1085.48399237, 1095.372864, 1105.07626751, 1115.18984928, 1127.09242581,
                        1138.79673007, 1149.43864103, 1159.42615783, 1169.31067405, 1179.38222723, 1189.4464981,
                        1199.5333784, 1209.88668205, 1220.04015567, 1229.96503806, 1240.57338913, 1250.6362063,
                        1260.9653672,  1271.4195118,  1281.94965148,1291.7051642,  1300.62445043, 1309.61923874,
                        1318.81874937, 1327.10100597, 1333.64664519 ]
    angles_BPic20 = [-17.10792503, -18.04861444, -19.00754622, -19.94279229, -20.87967121, -21.83354603, -22.80021385,
                -23.7361827,  -24.67226941, -25.64268633, -26.66322205, -27.64449798, -28.60997872, -29.57157742,
                -30.54446954, -31.52492384, -32.49536969, -33.46763605, -34.46496023, -35.42006557, -36.40736831,
                -37.39081194, -38.40896634, -39.38236154, -40.37439087, -41.34595291, -42.32240888, -43.28713799,
                -44.25890071, -45.2366961, -46.19115248, -47.14049291]
    # Given a range (1.5*pi radians), make a discrete sign function that is evaluated in the spacing that the wavelength channels are distributed
    wlMin, wlMax, wlDistribution, wlDistrib_sin = np.min(wavelength_channels), np.max(wavelength_channels), [], []
    WLC_num = 39
    x_squares, y_squares, center_square, location, location_demo, Xdelay = 5, 5, [3, 3], (44, 88), (44, 125), diameter / 4
    working_count = 0
    #fig, axs = plt.subplots(nrows=2, ncols=WLC_num, figsize=(8*WLC_num, 14))# sharey=False)
    for i in range(WLC_num):
        wlDistribution.append(10*(wlMax - wavelength_channels[i])/(wlMax - wlMin) - 7.6) # This will replace 39 WLCs with 1 period, which still needs to move ~10 radians so 10rad/1wlDistribution
        wlDistrib_sin.append(np.sin(wlDistribution[i]))
        wlD = wlDistribution[i]
        #checkerboard_demo, CB4D_D = make_oscillating_checkerboard_noGap(diameter, x_squares, y_squares, center_square, max_brightness, amp_nonRad_less, location_demo, Xdelay, wlD, decay_factor) # Ydelay2,
        #checkerboard_Ydelay, CB4D_Ydelay = make_oscillating_checkerboard_noGap(diameter, x_squares, y_squares, center_square, max_brightness, amp_nonRad_less, location, Xdelay, Ydelay, decay_factor)
        checkerboard, CB4D = make_oscillating_checkerboard_noGap(diameter, x_squares, y_squares, center_square, max_brightness, amp_nonRad_less, location, Xdelay, wlD, decay_factor) # Ydelay,
        #mitigated_signal_D = add_wavy_checkerboard(signalMatrix_FPI, checkerboard_demo)
        #mitigated_signal_Ydelay = add_wavy_checkerboard(signalMatrix_FPI, checkerboard_Ydelay)

        ### ### Align the CB, and then add it to the res_raw
        #print(np.shape(res_raw))  # (39, 32, 200, 200)!
        for j in range(len(angles_BPic20)):
            #print("\nj:"+str(j)+". -(j+1):"+str(-(j+1))+"\nangles_BPic20[j]:"+str(angles_BPic20[j])+"\nangles_BPic20[-j]"+str(angles_BPic20[-(j+1)]))
            #print("-1*angles_BPic20[j] should start with: -17.1, and is: "+str(-1*angles_BPic20[j]))     # yes... the angles are going in reverse order...
            #print("-1*angles_BPic20[-(j+1)] start with: -47.1, and is: "+str(-1*angles_BPic20[-(j+1)]))
            # Test: instead of angles = 60 and -60, try 60 and 300 or something positive... 10 Sep 2023
            aligned_CB, rotMat = align_CB_to_angle(checkerboard, -1*angles_BPic20[j])                   # a small-ish 50,50 matrix
            aligned_CB_neg, rotMat_neg = align_CB_to_angle(checkerboard, -1*angles_BPic20[-(j+1)])      # a small-ish 50,50 matrix

            '''     THESE WORKED! Separated by a set 120 degree difference
            print(np.shape(aligned_CB))
            print(np.shape(aligned_CB_neg))
            fig, axs = plt.subplots(nrows=1, ncols=2)#, figsize=(32*6, 32*0.625))
            axs[0].title.set_text('Testing the alignment of CBs in plots, j (exposure) is: '+str(j))
            axs[0].imshow(aligned_CB, origin='lower', cmap='Blues_r')
            axs[1].imshow(aligned_CB_neg, origin='lower', cmap='Blues_r')
            plt.show()
            '''

            #mitigated_signal = add_wavy_checkerboard(res_raw[i,j,:,:], aligned_CB)          # a 200,200 image
            #mitigated_signal_neg = add_wavy_checkerboard(res_raw[i,j,:,:], aligned_CB_neg)  # a 200,200 image
            mitigated_signal = add_wavy_checkerboard(np.zeros((200,200)), aligned_CB)          # The CB on a blank background
            mitigated_signal_neg = add_wavy_checkerboard(np.zeros((200,200)), aligned_CB_neg)  # The negative CB on a blank

            '''     THESE WORKED! Separated by an 120 degree input
            print(np.shape(mitigated_signal))
            print(np.shape(mitigated_signal_neg))
            fig, axs = plt.subplots(nrows=1, ncols=2)#, figsize=(32*6, 32*0.625))
            axs[0].title.set_text('Testing the mitigated signals in plots, j (exposure) is: '+str(j+1))
            axs[0].imshow(mitigated_signal, origin='lower', cmap='Blues_r')
            axs[1].imshow(mitigated_signal_neg, origin='lower', cmap='Blues_r')
            plt.show()
            '''

            '''     print shape of res raw,,, should be 200x200...?  ##### NEXT STEP HERE 8 SEP 2023!!! ###
            print('shape of res_raw[ij]', np.shape(res_raw[i,j,:,:]))           # 200x200
            print('shape of res_raw_neg[ij]', np.shape(res_raw_neg[i,j,:,:]))   # 200x200
            print('shape of mit_sig', np.shape(mitigated_signal))               # 200x200
            print('shape of mit_sig_neg', np.shape(mitigated_signal_neg))       # 200x200
            '''


            # Before: Print res_raw[i,j,:,:], mitigated_signal, res_raw_neg[i,j,:,:], and mitigated_signal_neg
            if j % 8 == 0 and i % 8 == 0:
                print('\n\n\tFIRST...\nres_raw[i,j,73:84,57:69]', res_raw[i,j,76:82,58:68])
                print('mitigated_signal[73:84,57:69]', mitigated_signal[76:82,58:68])
                print('res_raw_neg[i,j,73:84,57:69]', res_raw_neg[i,j,76:82,58:68])
                print('mitigated_signal_neg[73:84,57:69]', mitigated_signal_neg[76:82,58:68])

            # Assign values in the res_raw matrix
            res_raw[i,j,:,:] = mitigated_signal
            res_raw_neg[i,j,:,:] = mitigated_signal_neg # Check around here for errors! 8 Sep

            # After: Print res_raw[i,j,:,:], mitigated_signal, res_raw_neg[i,j,:,:], and mitigated_signal_neg


            counter_mitSig = 0
            counter = 0

            if j % 8 == 0 and i % 8 == 0:
                print('Check the values of the reduction at this WLC and exposure, i, j =', i, j)
                if len(mitigated_signal) != len(mitigated_signal_neg):
                    print('The matrices are different length... This is a problem to fix! (should I use shape instead?)')
                    counter_mitSig = 1
                for k in range(200):
                    for l in range(200):
                        #if res_raw[i,j,k,l] != res_raw_neg[i,j,k,l]:
                        if mitigated_signal[k,l] != mitigated_signal_neg[k,l]:
                            counter_mitSig = 1
                if counter_mitSig == 1:
                    print('mitSig & its negative are different (this is GOOD)')
                else:
                    print('mitSig & its negative are the same, which is BAD!')

                if len(res_raw[i,j,:,:]) != len(res_raw_neg[i,j,:,:]):
                    print('The matrices are different length... This is a problem to fix! (should I use shape instead?)')
                    counter = 1
                for k in range(200):
                    for l in range(200):
                        if res_raw[i,j,k,l] != res_raw_neg[i,j,k,l]:
                            counter = 1
                        if res_raw[i,j,k,l] > 16: # >8 yields ~12 points... >16 yields 5 points
                            print('High signal strength (>16) at coordinates k, l =', k, l)
                if counter == 1:
                    print('resRaw & its negative are different (this is GOOD)')
                else:
                    print('resRaw & its negative are the same, which is BAD!')
            #working_count += 1
            #print("it's working, res_raw & res_raw_neg are different...")
        #print('\n\tThe working_count (number of differences between regular and negative rotation) is:', working_count)
        #equal_matrix_test(res_raw, res_raw_neg)

        '''mitigated_signal_unaligned = add_wavy_checkerboard(res_raw[i,:,:,:], checkerboard)
        for j in range(len(angles)):
            mitigated_signal = align_CB_to_angle(mitigated_signal_unaligned[j,:,:], angles[j]) #maybe -j
            mitigated_signal_backwards = align_CB_to_angle(mitigated_signal_unaligned[j,:,:], angles[-j])
            #should edit the last two dimensions so that they are printing coordinate vales in the region the CB has been applied
            #print('\ni = '+str(i)+'. Before the res_raw[i,8,99:101,99:101] =\n'+str(res_raw[i,8,99:101,99:101])+'.')
            res_raw[i,j,:,:] = mitigated_signal
            res_raw_backwards[i,j,:,:] = mitigated_signal_backwards
            #print('Now the res_raw[i,8,99:101,99:101] =\n'+str(res_raw[i,8,99:101,99:101])+'.')
        '''

        # Make 2 subplots... 1 is of res_raw, 1x4 of WLC=8, and exposures={4, 12, 20, 28}. 2 is of the mitigated

    # outside the for loop, plot the two subplots
    '''fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(36, 18))
    axs[0].imshow(res_raw[8, 4,:,:], origin='lower', cmap='Blues_r')
    axs[1].imshow(res_raw[8,12,:,:], origin='lower', cmap='Blues_r')
    axs[2].imshow(res_raw[8,20,:,:], origin='lower', cmap='Blues_r')
    axs[3].imshow(res_raw[8,28,:,:], origin='lower', cmap='Blues_r')
    plt.show()
    '''

    print('\tFinished running the \'for i in range(WLC_num):\' for loop!')

    print("\n\n\tbefore modification, np.shape(res_raw):", shapeResRaw)
    print("\n\tbefore modification, np.shape(res_rot):", shapeResRot)
    print("\n\n\tafter modification, np.shape(res_raw):", np.shape(res_raw))
    print("\n\tafter modification, np.shape(res_rot):", np.shape(res_rot))


    fig, axs = plt.subplots(nrows=3, ncols=32, figsize=(32*6, 32*0.625))
    for i in range(32): # range(len(res_raw_original[0,:,0,0]))
        axs[0, i].title.set_text("exposure #"+str(i+1))#+". min: "+str(np.round(np.min(mitigated_signal), 2)) +". max: "+str(np.round(np.max(mitigated_signal), 2)) )
        axs[0, i].imshow(res_raw_original[11, i,:,:], origin='lower', cmap='Blues_r')
        axs[1, i].imshow(res_raw[11, i,:,:], origin='lower', cmap='Blues_r')
        axs[2, i].imshow(res_raw_neg[11, i,:,:], origin='lower', cmap='Blues_r')

    #fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(36, 18))
    #axs[0].imshow(res_raw[8, 4,:,:], origin='lower', cmap='Blues_r')
    #axs[1].imshow(res_raw[8,12,:,:], origin='lower', cmap='Blues_r')
    #axs[2].imshow(res_raw[8,20,:,:], origin='lower', cmap='Blues_r')
    #axs[3].imshow(res_raw[8,28,:,:], origin='lower', cmap='Blues_r')

    plt.show()


    ''' current thought, or state of the problem
    I have modified the raw images,
    I don't think the program has any code to then de-rotate and stack these modified images.
    That is, the rest of the reduction is using
    the res_rot that it calculated at the end of the original postproc.py file/code and
    doesn't recalculate the res_rot from the new res_raw I made by
    modifying the code after the end of the new postproc.py file/code. Is this the issue? I '''



    ''' CODE APPENDIX (future trash)

    if rotMat.all() == rotMat_neg.all():
                print('The rotation matrices are not working... The positive & negative are the same')
            if aligned_CB.all() != aligned_CB_neg.all():
                print('It is working... the positive and negative angle alignments make different CB')
    '''


    return res_raw, res_rot


In [ ]:
''' THE CODE APPENDIX

20 Aug 2023: Use this code/mitigation technique in/for/with the postproc.py file, use Jupyter.

# compare the Ydelays of the original and with the new wlDistribution, now they are close... 2.4 & 2.4 to -7.5 and -7.6
WLC_num = 39
PCA_1, PCA_2 = 1, 1
x_squares, y_squares, center_square, location, location_demo = 5, 5, [3, 3], (44, 88), (44, 125)
fig, axs = plt.subplots(nrows=2, ncols=WLC_num, figsize=(8*WLC_num, 14))# sharey=False)
Ydelay_old = []
Ydelay_new = []
for i in range(WLC_num):
    x, y = image_data_FPI[0][0][0], image_data_FPI[0][0][1]
    if len(np.shape(image_data_FPI)) == 5:
        signalMatrix_FPI = set_signalMatrix_1_WLC_2_PCA(image_data_FPI, x, y, PCA_1, PCA_2, i)   #elif len(np.shape(image_data_FPI)) == 4:   #signalMatrix_FPI = set_signalMatrix_1_WLC_1_PCA(image_data_FPI, x, y, 2, 32)
    else:
        print('Error... image_data_FPI != 5\n') #4 or
    Ydelay = (i * -0.26) + 2.4 # The black starts at the top, goes to the bottom and back to the top (1 cycle), and then to the bottom again (maybe a little more), so about 1.5 cycles, or 1.6-1.7... That would be 2*pi*1.6 = 10 radians over 39 WLC = 0.26rad/WLC...
    Ydelay2= (i * -0.26) + 2.4 # this moves the CB's black (wave) down. The observation's black kinda moves down (or the white up)
    Xdelay = diameter / 4
    Ydelay_old.append(Ydelay)
    Ydelay_new.append(wlDistribution[i])
print("Ydelay_old", Ydelay_old)
print("Ydelay_new", Ydelay_new)

'''